In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelBinarizer

import os

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
DATA_PATH = '../../../../Data/Home Credit Default Risk/'

print(os.listdir(DATA_PATH))

FileNotFoundError: [WinError 3] The system cannot find the path specified: '../../../../Data/Home Credit Default Risk/'

In [ ]:
app_train = pd.read_csv(DATA_PATH + '/application_train.csv')

In [ ]:
print('Training data shape: ', app_train.shape)
app_train.head()

In [ ]:
app_test = pd.read_csv(DATA_PATH + '/application_test.csv')

In [ ]:
print('Testing data shape: ', app_test.shape)
app_test.head()

Exploratory Data Analysis

In [ ]:
app_train['TARGET'].value_counts()

In [ ]:
app_train['TARGET'].astype(int).plot.hist()

Examine missing values

In [ ]:
len(app_train)

In [ ]:
def get_missing_values_table(df):
    missing_values_count_per_column_array = df.isnull().sum()
    
    rows_count = df.shape[0]
    columns_count = df.shape[1]
    
    missing_values_percent_per_column_array = 100 * missing_values_count_per_column_array / rows_count
    
    # create new table with number of missing values count and percentage for each column
    missing_values_table = pd.concat([missing_values_count_per_column_array, missing_values_percent_per_column_array], axis = 1)
    
    # rename the columns
    missing_values_table = missing_values_table.rename(columns = {0:'Missing Values', 1:'% of Total Values'})
    
    # leave only the columns that have missing values
    missing_values_table = missing_values_table[missing_values_table.iloc[:, 1] != 0]
    
    # sort descending by percentage and round to 1 decimal places
    missing_values_table = missing_values_table.sort_values('% of Total Values', ascending=False).round(1)
    
    number_of_columns_with_missing_values = missing_values_table.shape[0]

    # print info
    print(str(number_of_columns_with_missing_values) + " of " + str(columns_count) + " have missing values")
    
    # return the dataframe with missing information
    return missing_values_table

In [ ]:
# missing values for training data
missing_values_train_data = get_missing_values_table(app_train)
missing_values_train_data.head(40)

Research the types of columns

In [ ]:
app_train.dtypes.value_counts()